# ML Trader

In [1]:
from utils.packages import *
from utils.variables import *
from utils.i_o import *
from utils.dir_slope import *
from utils.engine import *
from utils.loops import *
from utils.order import *
from utils.ml import *
import winsound

# Notes
---
## 2022_09_02

### Observations

1. No need to make 10 transactions a day. Just one is enough

1. lema or sema match
    1. Using lema match works fine in both opening 
    1. Using lema has better results
    1. 144 produces better resuls over higher numbers like 1440 

1. Lema angle in order and close
    1. Using lema angle in order, delays order, but overall profits are high
    1. Using lema angle in close also increases overall profit

1. Before everything need to find why 2021 is consistently producing negative results in all the setups.
    1. What is that this causing this issue, how can it be addressed with in the same system
    1. less complex method is essential
    

### To Test
1. Find whether the opening or closing that is causing the most issue in loss years
1. Test other years to understand how well this setup works.
1. Remove lema match check from closing. This is delaying bad trasactions
1. Test 5 round up on lema match. Transactions are opening in lema match gaps
1. Ideation
    1. Test lema Bollinger bands to understand flow direction
    1. Test overall flow direction

---

## 2022_09_02

### Observations
1. Opening and closing need to happen faster. Cant close after ema match
1. Lema angle helps save positions against frequent flips but delays closing and causes losses
1. Market is non-directional (Dec 2021 is a good example) Goes up and down, when algo tries to open and close its always too late
1. Need a faster method


### To Test
1. Should test plotting close as a line instead of candles. Perform match in close with wider window
1. Close chart needs to be analysed. This looks promising.
1. Go with simple rolling min max for close. Works better than resample min max merge

---


## 2022_09_08

### Observations
1. Analyse sema gap window in line with other metrics. 


---

## Result by Years

| Year | Angle open | Angle close | Lema match open | Lema match close | Result type | pips  |
| ---- | ---------- | ----------- | --------------- | ---------------- | ----------- | ----- |
| 2022 | Yes        | No          | Yes             | Yes              | Profit      | 577   |
| 2021 | Yes        | No          | Yes             | Yes              | Loss        | \-596 |
| 2020 | Yes        | No          | Yes             | Yes              | Profit      | 1462  |
| 2019 | Yes        | No          | Yes             | Yes              | Loss        | 193   |
| 2018 | Yes        | No          | Yes             | Yes              | Profit      | 60    |
| 2017 | Yes        | No          | Yes             | Yes              | Profit      | \-1396|    

data                = get_date_list(data)
data                = read_data(data)

In [2]:
data["input_year"] = 2021
data['start_date'] = {'year':data["input_year"], 'month':1, 'date':1}
data['end_date']   = {'year':data["input_year"], 'month':12, 'date':31}
data["product"] = 'eurusd'
data["input_file_name"] = f'{data["product"]}_{data["input_year"]}.csv'

In [3]:
%%time
data = run_engine(data)

pl   -0.00302
dtype: float64

100%|██████████████████████████████████| 57586/57586 [00:17<00:00, 3365.68it/s]


--------------------------------------
            DateTime_frmt  order_side  close_type       pl
55780 2021-12-23 15:25:00         NaN  sema_close -0.00177
56289 2021-12-27 09:55:00         NaN  sema_close -0.00126
56291 2021-12-27 10:05:00         NaN  sema_close -0.00091
56320 2021-12-27 12:30:00         NaN  sema_close -0.00089
56339 2021-12-27 14:05:00         NaN  sema_close -0.00065
56638 2021-12-28 15:00:00         NaN  sema_close -0.00133
56918 2021-12-29 14:20:00         NaN  sema_close -0.00256
57137 2021-12-30 08:35:00         NaN  sema_close -0.00190
57171 2021-12-30 11:25:00         NaN  sema_close -0.00161
57255 2021-12-30 18:25:00         NaN  sema_close -0.00177
57274 2021-12-30 20:00:00         NaN  sema_close -0.00220
57403 2021-12-31 06:45:00         NaN  sema_close -0.00171
57406 2021-12-31 07:00:00         NaN  sema_close -0.00052
57409 2021-12-31 07:15:00         NaN  sema_close -0.00060
57432 2021-12-31 09:10:00         NaN  sema_close -0.00175



100%|█████████████████████████████████████████| 91/91 [00:00<00:00, 392.67it/s]

Wall time: 1min 44s


In [4]:
data['df_ohlc'].groupby(['close_type'])['pl'].sum()

close_type
sema_close   -0.00302
Name: pl, dtype: float64

In [5]:
x = data['df_ohlc'].groupby(['month_val'])['pl'].sum()
print(x)
print('---------------------')
print(x.sort_values())
print('---------------------')
print(f'max : {x.idxmax()} : {max(x)}')
print(f'min : {x.idxmin()} : {min(x)}')
print('---------------------')
monthly_avg = np.round(data['df_ohlc']['pl'].sum()/len(set(data['df_ohlc']['month_val'])),4)
print(f'monthly_avg : {monthly_avg}')

month_val
3     0.00000
4    -0.00573
5     0.01889
6    -0.01524
7     0.03127
8    -0.00853
9    -0.01502
10   -0.00087
11    0.00000
12   -0.00779
Name: pl, dtype: float64
---------------------
month_val
6    -0.01524
9    -0.01502
8    -0.00853
12   -0.00779
4    -0.00573
10   -0.00087
3     0.00000
11    0.00000
5     0.01889
7     0.03127
Name: pl, dtype: float64
---------------------
max : 7 : 0.03127
min : 6 : -0.01524
---------------------
monthly_avg : -0.0003


In [6]:
sys.exit()

SystemExit: 

In [ ]:
print(data['df_small'].groupby(['weekday', 'pl_type'])['pl_type'].count().unstack())
print('----------------------------------------------')
print(data['df_small'].groupby(['weekday', 'pl_type'])['pl'].sum().unstack())

In [ ]:
data['lema_gap_pl_analysis_month'] = 3

data = lema_gap_pl_analysis(data)
data['lema_gap_pl_report']

In [ ]:
y = data['df_ohlc']['pl'].sum()
x = data['df_ohlc'].groupby(['month_val'])['pl'].sum()
send_telegram_message(f'{data["input_year"]}')
send_telegram_message(f'{y}')
send_telegram_message(f'{x}')
send_telegram_message(f'-------------------')

In [ ]:
y = data['df_ohlc'][data['df_ohlc']['month_val'] == 1]
x = y.groupby(['date_val'])['pl'].sum()
print(x)
print('---------------------')
print(f'max : {x.idxmax()} : {max(x)}')
print(f'min : {x.idxmin()} : {min(x)}')
print('---------------------')

In [ ]:
data['df_ohlc'].tail()

In [ ]:
data['df_ohlc']['lema_gap'].describe()

In [ ]:
x = 5
data['df_ohlc'].iloc[-5:,x:x+15]

In [ ]:
data['df_small']['lema_gap'].describe()

In [ ]:
a = np.arange(0,20)
for b in a:
    x = int(round(b/5.0)*5.0)
    print(b, x)

In [ ]:
abs(all['negative_sum']) / all['positive_sum']

In [ ]:
df_small['order_side']

In [ ]:
data['df_small'].to_csv('data/temp.csv')

In [7]:
data["plot"] = True
data['plot_type'] = 'file'
data['reduce_plot'] = True
chart_duration = 'year'
month = 12

# data['things_to_plot'] = ['tick', 'ema', 'BBands', 'dots']
# data['things_to_plot'] = ['ema', 'BBands', 'dots', 'indicators']
# data['things_to_plot'] = ['ema', 'BBands', 'dots', 'cross']
# data['things_to_plot'] = ['ema', 'BBands', 'dots', 'positions', 'tip']
# data['things_to_plot'] = ['ema', 'BBands', 'dots', 'positions', 'lema_angle']
# data['things_to_plot'] = ['ema', 'sema_angle', 'slema_angle', 'lema_angle', 'dir']
# data['things_to_plot'] = ['ema', 'positions', 'lema_angle','slema_angle','sema_angle', 'rsi']
# data['things_to_plot'] = ['ema','positions', 'lema_angle','lema_diff', 'lema_max_min']
# data['things_to_plot'] = ['candles','sema', 'slema','positions','lema', 'lema_angle', 'lema_max_min', 'close_diff']
# data['things_to_plot'] = ['lema', 'tip', 'candles', 'lema_angle', 'lema_change']
# data['things_to_plot'] = ['candles','sema', 'close','lema', 'lema_match', 'positions', 'lema_change']
# data['things_to_plot'] = ['close', 'sema','lema', 'lema_change', 'lema_match', 'close_max']
# data['things_to_plot'] = ['close', 'tip','lema', 'close_max', 'close_min', 'lema_match']
# data['things_to_plot'] = ['close', 'lema','lema_match', 'slema', 'positions']
# data['things_to_plot'] = ['close', 'sema', 'slema','lema','lema_match', 'tip']
# data['things_to_plot'] = ['close', 'sema','lema','lema_match', 'match_max', 'match_min', 'unmatched']
# data['things_to_plot'] = ['close', 'sema','lema','lema_match', 'lema_max_min', 'positions']
data['things_to_plot'] = ['close','lema', 'slema', 'sema', 'positions']

data['plot_start']  = f"{data['input_year']}-{month}-1"

if chart_duration == 'month':
    if month == 12:
        data['plot_stop']   = f"{data['input_year']+1}-1-1"
    else:
        data['plot_stop']   = f"{data['input_year']}-{month+1}-1"

elif chart_duration == 'year':        
    data['plot_start']  = f"{data['input_year']}-1-1"
    data['plot_stop']   = f"{data['input_year']+1}-1-1"


plot_graph(data)

generate_result_report(data)

In [ ]:
cols = ['DateTime_frmt', 'close', 'ask','bid', 
 'open', 'high', 'low', 'candle_size', 'lema', 'lema_angle', 'lema_angle_2',
       'lema_angle_0', 'slema', 'slema_angle', 'slema_angle_2', 'sema',
       'sema_angle', 'sema_angle_2', 'close_diff', 'BBand_upper',
       'BBand_middle', 'BBand_lower', 'BBand_width', 'avg_BBand_width', 'adx',
       'tip', 'lema_match', 'order_side', 'order_size', 'long_open',
       'short_open', 'close_type', 'pl', 'up', 'down', 'month_val', 'date_val',
       'hour_val', 'long_profit', 'long_loss', 'short_profit', 'short_loss']

data['df_ohlc'][cols].to_csv('data/temp.csv')

In [ ]:
data['df_ohlc']['lema_match'][17866-25:17866]

In [ ]:
l = pd.Series([1,2,3,np.nan, 4,5])
l

print(l.fillna(value=0).rolling(2).max())
max(l)

In [ ]:
data['df_ohlc']['match_max'] = data['df_ohlc']['lema_match'].fillna(value=0).rolling(window = data['look_back_window_size']).max()
data['df_ohlc']['match_min'] = data['df_ohlc']['lema_match'].fillna(value=0).rolling(window = data['look_back_window_size']).min()

data['df_ohlc'].loc[data['df_ohlc']['match_max'] == 0, 'match_max'] = np.nan
data['df_ohlc'].loc[data['df_ohlc']['match_min'] == 0, 'match_min'] = np.nan